In [1]:
import numpy as np
import pandas as pd

In [47]:
df = pd.read_csv('/content/All countries Important Indicators for MVI.csv', encoding='latin-1')

In [48]:
null_counts = df.isnull().sum().sum()
print(f"Total null values in the DataFrame: {null_counts}")


Total null values in the DataFrame: 23228


In [49]:
from sklearn.linear_model import LinearRegression

def impute_highly_correlated(df,
                             year_cols=[str(y) for y in range(2010,2025)],
                             country_col='Country Code',
                             series_col='Series Name',
                             threshold=0.9):
    """
    df: DataFrame with columns
        ['Country Name','Country Code','Series Name', '2010','2011',…,'2024'].
    year_cols: list of the year-column names (strings).
    threshold: abs(corr) above which we regress & impute.
    """
    df_out = df.copy()
    lr = LinearRegression()

    for cc in df[country_col].unique():
        sub = df[df[country_col]==cc]
        # pivot to wide: index=Series Name, columns=years
        wide = sub.set_index(series_col)[year_cols]

        # compute |corr| between series (corr across years)
        corr = wide.T.corr().abs()

        # for each pair (X,Y) with high corr
        for X in corr.columns:
            for Y in corr.columns:
                if X >= Y:
                    continue
                if corr.loc[X,Y] <= threshold:
                    continue

                x = wide.loc[X]
                y = wide.loc[Y]
                ok = x.notna() & y.notna()
                if ok.sum() < 2:
                    continue

                # fit X = alpha + beta * Y
                lr.fit(y[ok].values.reshape(-1,1), x[ok].values)
                α, β = lr.intercept_, lr.coef_[0]

                # find years where X missing but Y present
                miss = x.isna() & y.notna()
                if not miss.any():
                    continue

                # impute
                years_to_fill = miss[miss].index
                wide.loc[X, years_to_fill] = α + β * y.loc[years_to_fill]

        # write back into df_out
        # for each series/year, update the corresponding cell
        for series in wide.index:
            for yr in year_cols:
                df_out.loc[
                    (df_out[country_col]==cc) &
                    (df_out[series_col]==series),
                    yr
                ] = wide.at[series, yr]

    return df_out

# Example usage:
df_filled = impute_highly_correlated(df, threshold=0.9)
null_counts = df_filled.isnull().sum().sum()
print(f"Total null values in the DataFrame: {null_counts}")

Total null values in the DataFrame: 20565


In [50]:
def impute_with_interpolation(df, year_cols):
    """Imputes NaN values in a DataFrame using linear interpolation.

    Args:
        df: The input DataFrame with time series data.
        year_cols: A list of column names representing years.

    Returns:
        A DataFrame with NaN values imputed using linear interpolation.
    """

    df_imputed = df.copy()  # Create a copy to avoid modifying the original DataFrame

    for index, row in df_imputed.iterrows():
        # Extract time series values for the current row
        time_series = row[year_cols].astype(float)

        # Interpolate missing values using linear interpolation
        time_series = time_series.interpolate(method='linear', limit_direction='both')

        # Update the row in the imputed DataFrame
        df_imputed.loc[index, year_cols] = time_series

    return df_imputed

# Example usage (assuming df_filled is your DataFrame and year_cols are defined):
year_cols = [str(year) for year in range(2019, 2025)]
df_filled = impute_with_interpolation(df_filled, year_cols)

null_counts = df_filled.isnull().sum().sum()
print(f"Total null values in the DataFrame after interpolation: {null_counts}")


Total null values in the DataFrame after interpolation: 16378


In [51]:
import pandas as pd

# 1. Define year columns and keep first three cols
year_cols = [str(year) for year in range(2020, 2025)]
first_three_cols = df_filled.columns[:3].tolist()

# 2. Subset to only the relevant columns
filtered_df = df_filled[first_three_cols + year_cols]

# 3. For each Series Name, fill NaNs in each year with that series’ mean
filled_df = filtered_df.copy()
filled_df[year_cols] = (
    filtered_df
    .groupby('Series Name')[year_cols]
    .transform(lambda col: col.fillna(col.mean()))
)

null_counts = filled_df.isnull().sum().sum()
print(f"Total null values in the DataFrame after interpolation: {null_counts}")

Total null values in the DataFrame after interpolation: 0


In [53]:
import pandas as pd

# 1. List of 'bad' series whose higher raw values imply more fragility (to be inverted)
bad_series = [
    'Central government debt, total (% of GDP)',
    'External debt stocks (% of GNI)',
    'Short-term debt (% of total reserves)',
    'Gross portfolio equity liabilities to GDP (%)',
    'Outstanding international private debt securities to GDP (%)',
    'Inflation, consumer prices (annual %)',
    'Unemployment, total (% of total labor force) (national estimate)',
    'Current account balance (% of GDP)'
]

# Build a polarity map for quick lookup
polarity = {
    name: ('bad' if name in bad_series else 'good')
    for name in filled_df['Series Name'].unique()
}


# 3. Function to normalize one group (one series) across all countries & years
def norm_group(df_group: pd.DataFrame) -> pd.DataFrame:
    """
    For each year column in this series group:
    - Compute min/max across all countries in the group
    - Apply (x - min)/(max - min) if 'good', or (max - x)/(max - min) if 'bad'
    """
    name = df_group.name
    pol = polarity[name]
    df_out = df_group.copy()
    for col in year_cols:
        mn, mx = df_group[col].min(), df_group[col].max()
        if mn == mx:
            # If no variation, set midpoint to avoid division by zero
            df_out[f"{col}_norm"] = 0.5
        else:
            if pol == 'good':
                df_out[f"{col}_norm"] = (df_group[col] - mn) / (mx - mn)
            else:
                df_out[f"{col}_norm"] = (mx - df_group[col]) / (mx - mn)
    return df_out

# 4. Apply to each series (group_keys=False preserves original row order)
filled_df_norm = (
    filled_df
      .groupby('Series Name', group_keys=False)
      .apply(norm_group)  # split-apply-combine via .apply :contentReference[oaicite:0]{index=0}
)
weights = {
    '2020_norm': 1,
    '2021_norm': 2,
    '2022_norm': 3,
    '2023_norm': 4,
    '2024_norm': 5,
}

# Compute the sum of weights (1+2+3+4+5 = 15)
total_weight = sum(weights.values())

# Compute the weighted average across the normalized columns
filled_df_norm['final_norm'] = (
    filled_df_norm[list(weights.keys())]            # select the five norm columns
    .mul(pd.Series(weights))                       # multiply each column by its weight
    .sum(axis=1) / total_weight                    # sum across columns, then divide by total weight
)

   2020_norm  2021_norm  2022_norm  2023_norm  2024_norm
0   0.667063   0.724395   0.724704   0.705413   0.704931
1   0.106272   0.164186   0.157831   0.187459   0.187459
2   0.647209   0.633830   0.573001   0.558342   0.558342
3   0.804243   1.000000   1.000000   0.270479   0.270479
4   0.968829   0.943835   0.942851   0.592494   0.592494


<ipython-input-53-c3374d1f48e0>:48: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(norm_group)  # split-apply-combine via .apply :contentReference[oaicite:0]{index=0}


In [56]:
df_for_ranking = filled_df_norm.iloc[:, [0, 1, 2, -1]]
df_for_ranking

,Country Name,Country Code,Series Name,final_norm
0,Afghanistan,AFG,"Central government debt, total (% of GDP)",0.709085
1,Afghanistan,AFG,Control of Corruption: Estimate,0.173018
2,Afghanistan,AFG,Current account balance (% of GDP),0.577263
3,Afghanistan,AFG,Current health expenditure (% of GDP),0.549237
4,Afghanistan,AFG,External debt stocks (% of GNI),0.734500
...,...,...,...,...
4546,Zimbabwe,ZWE,"Foreign direct investment, net inflows (% of GDP)",0.297107
4547,Zimbabwe,ZWE,Total reserves (% of total external debt),0.000383
4548,Zimbabwe,ZWE,Short-term debt (% of total reserves),0.000000
4549,Zimbabwe,ZWE,Gross portfolio equity liabilities to GDP (%),0.909266


In [57]:
import pandas as pd

# 1. Compute rank within each series (1 = highest final_norm)
df_ranked = filled_df_norm.copy()
df_ranked['Rank'] = (
    df_ranked
      .groupby('Series Name')['final_norm']
      .rank(method='min', ascending=False)
)

# 2. Pivot to wide format: one row per country, one col per series
df_pivot = (
    df_ranked
      .pivot_table(
          index=['Country Name','Country Code'],
          columns='Series Name',
          values='Rank'
      )
      .reset_index()
)

# 3. Rename the series columns to append "_Rank"
df_pivot.columns = [
    f"{col}_Rank" if col not in ['Country Name','Country Code'] else col
    for col in df_pivot.columns
]

# Now df_pivot has: Country Name, Country Code, and <Series Name>_Rank for every series
df_pivot.head()

,Country Name,Country Code,"Central government debt, total (% of GDP)_Rank",Control of Corruption: Estimate_Rank,Current account balance (% of GDP)_Rank,Current health expenditure (% of GDP)_Rank,External debt stocks (% of GNI)_Rank,"Foreign direct investment, net inflows (% of GDP)_Rank",GDP growth (annual %)_Rank,GDP per capita (current US$)_Rank,...,Human capital index (HCI) (scale 0-1)_Rank,"Inflation, consumer prices (annual %)_Rank",Outstanding international private debt securities to GDP (%)_Rank,Political Stability and Absence of Violence/Terrorism: Estimate_Rank,Regulatory Quality: Estimate_Rank,Research and development expenditure (% of GDP)_Rank,Short-term debt (% of total reserves)_Rank,Total reserves (% of total external debt)_Rank,"Unemployment, total (% of total labor force) (national estimate)_Rank",Voice and Accountability: Estimate_Rank
0,Afghanistan,AFG,51.0,191.0,15.0,2.0,14.0,190.0,215.0,216.0,...,179.0,3.0,67.0,214.0,197.0,40.0,2.0,3.0,100.0,212.0
1,Albania,ALB,199.0,134.0,75.0,123.0,72.0,27.0,48.0,113.0,...,79.0,57.0,67.0,96.0,83.0,40.0,43.0,138.0,187.0,95.0
2,Algeria,DZA,51.0,154.0,170.0,179.0,3.0,180.0,135.0,138.0,...,140.0,127.0,67.0,175.0,188.0,40.0,16.0,1.0,113.0,172.0
3,American Samoa,ASM,51.0,25.0,93.0,88.0,81.0,47.0,183.0,84.0,...,87.0,160.0,NaN,17.0,61.0,40.0,81.0,26.0,113.0,45.0
4,Andorra,AND,51.0,25.0,207.0,63.0,81.0,47.0,43.0,33.0,...,87.0,160.0,67.0,4.0,21.0,40.0,81.0,26.0,113.0,38.0


In [58]:
numerical_cols = df_pivot.select_dtypes(include=np.number).columns
df_pivot['final_rank'] = df_pivot[numerical_cols].sum(axis=1)

AttributeError: 'DataFrame' object has no attribute 'sort_by'

In [62]:
# These are not fully independent UN member states, but overseas or autonomous territories whose data collection and institutional contexts differ. To remove these countries code ABW, BMU, VGB, CYM, CUW, PYF, GIB, GRL,
# GUM, HKG, MAC, IMN, MNP, PRI, VIR, SXM,
# MAF, CHI, TCA, FRO, NCL, STP, CUW from the final ranking

# List of country codes to remove
countries_to_remove = ['ABW', 'BMU', 'VGB', 'CYM', 'CUW', 'PYF', 'GIB', 'GRL',
                       'GUM', 'HKG', 'MAC', 'IMN', 'MNP', 'PRI', 'VIR', 'SXM',
                       'MAF', 'CHI', 'TCA', 'FRO', 'NCL', 'STP', 'CUW']

# Remove the specified countries from the DataFrame
df_pivot = df_pivot[~df_pivot['Country Code'].isin(countries_to_remove)]


In [66]:
# 1. Sort by the existing final_rank score (ascending: best first)
final_ranking = df_pivot[['Country Name', 'Country Code', 'final_rank']] \
                   .sort_values('final_rank', ascending=False)

# 2. Reset the index (dropping the old one)
final_ranking = final_ranking.reset_index(drop=True)

# 3. Overwrite `final_rank` with the actual rank positions (1 through N)
final_ranking['final_rank'] = final_ranking.index + 1

# 4. (Optional) Rename for clarity
final_ranking = final_ranking.rename(columns={'final_rank': 'Rank'})

In [69]:
final_ranking.to_csv('MVI rankings.csv', index=False)

In [68]:
final_ranking

,Country Name,Country Code,Rank
0,Angola,AGO,1
1,Zimbabwe,ZWE,2
2,"Congo, Rep.",COG,3
3,"Yemen, Rep.",YEM,4
4,Syrian Arab Republic,SYR,5
...,...,...,...
194,New Zealand,NZL,195
195,"Korea, Dem. Rep.",PRK,196
196,Turkey,TUR,197
197,Czech Republic,CZE,198
